# Environmental Shock

In [8]:
import numpy as np
import scipy as sp
from scipy.optimize import minimize
import pandas as pd
from scipy.optimize import fsolve
import copy 
import ipynb.fs.defs.Functions_Equations_Equilibrium_Simulation as baseFuncs

In [2]:
def simulate_behavioral_shock(base_para, shock_para, T = 100, shock_times = [10, 20], process_removal_proportion = None, start_from = None):
    # Start from equilibrium
    para = base_para
    if start_from is None:   
        res, _, _, _ = baseFuncs.equilFuncBehavioral(para)
        para['init_R_u'] = res[0]
        para['init_R_o'] = res[1]
    else:
        para['init_R_u'] = start_from[0]
        para['init_R_o'] = start_from[1]
    
    # Simulate before, during, and after shock    
    R_us1, R_os1, w_As1, w_Ds1, Us1, discUs1, timesteps1 = baseFuncs.simulate(para, tau = shock_times[0], T = shock_times[0], generalAlloc = False)
    para = shock_para
    if process_removal_proportion is not None:
        para['init_R_u'] = R_us1[-1]*(1-process_removal_proportion)
        para['init_R_o'] = R_os1[-1]*(1-process_removal_proportion)
    else:
        para['init_R_u'] = R_us1[-1]
        para['init_R_o'] = R_os1[-1]

    
    if shock_times[1] is not None:
        R_us2, R_os2, w_As2, w_Ds2, Us2, discUs2, timesteps2 = baseFuncs.simulate(para, tau = shock_times[1]-shock_times[0], T = shock_times[1]-shock_times[0], generalAlloc = False)
        para = base_para
        para['init_R_u'] = R_us2[-1]
        para['init_R_o'] = R_os2[-1]
        R_us3, R_os3, w_As3, w_Ds3, Us3, discUs3, timesteps3 = baseFuncs.simulate(para, tau = T-shock_times[1], T = T-shock_times[1], generalAlloc = False)
        
        # Merge all results
        R_us = R_us1 + R_us2 + R_us3
        R_os = R_os1 + R_os2 + R_os3
        w_As = w_As1 + w_As2 + w_As3
        w_Ds = w_Ds1 + w_Ds2 + w_Ds3
        Us = Us1 + Us2 + Us3
        discUs = discUs1 + discUs2 + discUs3
        timesteps = timesteps1 + list(np.asarray(timesteps2)+shock_times[0]) + list(np.asarray(timesteps3)+shock_times[1])
        rds = list(np.ones(len(R_us1))*base_para['r_d']) + list(np.ones(len(R_us2))*shock_para['r_d']) + list(np.ones(len(R_us3))*base_para['r_d'])
    else:
        R_us2, R_os2, w_As2, w_Ds2, Us2, discUs2, timesteps2 = baseFuncs.simulate(para, tau = T-shock_times[0], T = T-shock_times[0], generalAlloc = False)
        R_us = R_us1 + R_us2
        R_os = R_os1 + R_os2
        w_As = w_As1 + w_As2
        w_Ds = w_Ds1 + w_Ds2
        Us = Us1 + Us2
        discUs = discUs1 + discUs2
        timesteps = timesteps1 + list(np.asarray(timesteps2)+shock_times[0])
        rds = list(np.ones(len(R_us1))*base_para['r_d']) + list(np.ones(len(R_us2))*shock_para['r_d'])

    return R_us, R_os, w_As, w_Ds, Us, discUs, timesteps, rds